# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-02-events-Copy1.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-ev

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8227


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6976


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [1]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

"select row.artist,row.song,row.lenght from music_app where sessionId = '338' and itemInSession = '4'"
"select artist,song,concat(firstName,lastName) from music_user where userId = '10' and sessionId = '182'"
"select concat(firstName,lastName) from music_app where song = 'All Hands Against His Own'"

                    

In [9]:
query = "CREATE TABLE IF NOT EXISTS music_app"
query = query + "(artist text,itemInSession int,length float,sessionId int,song text, PRIMARY KEY(artist,song,length,itemInSession,sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
query = "CREATE TABLE IF NOT EXISTS music_user"
query = query + "(artist text,firstName text,lastName text,sessionId int,itemInSession int,userId int,song text,PRIMARY KEY(artist,itemInSession,song,firstName,lastName,userId,sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
query = "CREATE TABLE IF NOT EXISTS music_song"
query = query + "(firstName text,lastName text,song text, PRIMARY KEY(firstName,lastName,song))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app (artist,itemInSession,length,sessionId,song)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]),line[9]))

In [13]:
        query = "INSERT INTO music_user (artist,firstName,lastName,sessionId,itemInSession,userId,song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to 'line[0], line[1]'
        session.execute(query, (line[0], line[1], line[4], int(line[8]), int(line[3]), int(line[10]), line[9]))

In [13]:
        query = "INSERT INTO music_song (firstName,lastName,song)"
        query = query + " VALUES (%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to 'line[0], line[1]'
        session.execute(query, (line[1], line[4], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query = "select artist,itemInSession,length,sessionId,song from music_app"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,row.iteminsession,row.length,row.sessionid,row.song)

Sheryl Crow 20 298.7881164550781 744 Picture
Wyclef Jean featuring Akon_ Lil Wayne_ and introducing Niia 32 239.4379119873047 574 Sweetest Girl (Dollar Bill)
Pat Green 48 295.3399353027344 172 Whiskey
Fanfarlo 19 270.78485107421875 605 I'm a Pilot
Pat Martino 10 355.94403076171875 297 El Hombre
Placebo 55 224.02566528320312 648 Breathe Underwater
Placebo 35 241.5277099609375 392 English Summer Rain
Placebo 0 324.02239990234375 280 Passive Aggressive
Placebo 23 254.45831298828125 129 Taste In Men
Placebo 79 190.6150665283203 984 The Bitter End
Sugababes 33 275.93096923828125 977 Overload
Sugababes 98 275.93096923828125 324 Overload
Delirious? 2 397.2697448730469 312 History Maker
Delirious? 2 250.95791625976562 264 Paint The Town Red
Gin Blossoms 0 233.48199462890625 679 Day Job
Corinne Bailey Rae 25 192.86158752441406 594 Another Rainy Day
Corinne Bailey Rae 11 206.1840362548828 548 Trouble Sleeping
Billie The Vision & The Dancers 4 136.46322631835938 1008 Still be Friends
Dragonforce 

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


query = "select artist,song,firstName,lastName,userId,userId from music_user where userId = '10' and sessionId = '182'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,row.song,row.firstname,row.lastname,row.userid,row.sessionid)

                    

Error from server: code=2200 [Invalid query] message="Invalid STRING constant (10) for "userid" of type int"


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

In [17]:
desc(music_user)

NameError: name 'desc' is not defined

### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [14]:
query = "DROP TABLE IF EXISTS music_app"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS music_user"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS music_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()